##### ANALITICO DE DISCAGENS

In [ ]:
from funcoes_claro import *
#################################################################################### CARREGANDO TODAS AS DISCAGENS DO MÊS #########################################################
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES CLARO\CHAMADAS_EQUIPE_CLARO\JULHO_24"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    lista.append(arquivo)
    dfList = []
    for item in lista:   
        chunks = pd.read_csv(item, chunksize=1000000, sep=';', encoding='iso-8859-1', low_memory=False, dtype={'TELEFONE': str})
        for df in chunks:
            dfList.append(df)
# Criando DataFrame com concat e a lista criada a cima
discagens_claro_full = pd.concat(dfList, sort=False, ignore_index=False)
discagens_claro_full = discagens_claro_full.reset_index(drop=True)

### somando os totais diarios para o numero real de discado, atendido, etc
discagens_claro_full = discagens_claro_full.groupby(['TELEFONE']).agg({
        'DISCADO_SIM':'sum',
        'ATENDIDO_SIM': 'sum',
        'DTMF_SIM': 'sum',
        'ATENDIDO_SIM':'sum'}).reset_index().sort_values(['DISCADO_SIM'])

#tratando a coluna TELEFONE
discagens_claro_full['TELEFONE'] = discagens_claro_full['TELEFONE'].astype(str).str.strip()

### DISCADOS UNICOS E ATENDIDOS UNICOS
discagens_claro_full.loc[discagens_claro_full["DISCADO_SIM"] >= 1.0, 'DISCADO_UNICO'] = 1
discagens_claro_full.loc[discagens_claro_full["ATENDIDO_SIM"] >= 1.0, 'ATENDIDO_UNICO'] = 1
discagens_claro_full['ATENDIDO_UNICO'] = discagens_claro_full['ATENDIDO_UNICO'].fillna(0)
discagens_claro_full = discagens_claro_full.reset_index(drop=True)


In [ ]:
### Carregando a base de telefones BLACKLIST
blacklist = pd.read_csv(r"R:\TI\TELEFONIA/BLACKLIST_NPTIM.csv",sep=';',dtype=str)
blacklist = blacklist.drop_duplicates(['dddtelefone']).reset_index(drop=True)

#tratando a coluna TELEFONE
blacklist['dddtelefone'] = blacklist['dddtelefone'].astype(str).str.strip()
blacklist = blacklist.rename(columns={'dddtelefone':'TEL_BLACKLIST'})
blacklist = blacklist.assign(BLACKLIST = 1 )

In [ ]:
### Carregando a base de telefones validados
telefones_validados = pd.read_csv(r"R:\TI\TELEFONIA\validados_2024/telefones_validados_full.csv",sep=';',dtype=str)
# telefones_validados = telefones_validados.assign(STATUS_TEL = 'Alto_Alô')
telefones_validados.rename(columns={'ResultadoClassificacao':'STATUS_TEL'},inplace=True)
telefones_validados['dddtelefone'] = telefones_validados['dddtelefone'].str.strip()

In [ ]:
########################################### CARREGANDO A BASE PADRONIZADA
def carregar_bases_claro_padronizada(caminho, extensao="txt", chunksize=1000000, sep=';', encoding='iso-8859-1'):
    try:
        # Encontrar todos os arquivos com a extensão especificada no caminho
        arquivos = glob.glob(f"{caminho}/*.{extensao}")
        if not arquivos:
            raise FileNotFoundError(f"Nenhum arquivo '{extensao}' encontrado no caminho especificado: {caminho}")
        
        df_list = []

        # Iterar sobre cada arquivo encontrado
        for arquivo in arquivos:
            print(f"Carregando {arquivo}...")
            # Ler o arquivo em chunks e adicionar à lista de DataFrames
            for chunk in pd.read_csv(arquivo, chunksize=chunksize, dtype='str', sep=sep, encoding=encoding):
                df_list.append(chunk)

        # Concatenar todos os DataFrames em um único DataFrame
        bases_concatenadas = pd.concat(df_list, sort=False, ignore_index=True)
        print("Carregamento e concatenação concluídos com sucesso.")
        return bases_concatenadas
    
    except Exception as e:
        print(f"Erro ao carregar arquivos: {e}")
        return None
    

############################################## CARREGANDO AS BASES UNIFICADAS CLARO ##########################################################
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO"
bases_claro_operacional = carregar_bases_claro_padronizada(caminho)

bases_claro_operacional['ATRASO'] = bases_claro_operacional['ATRASO'].astype('int16')
bases_claro_operacional['SALDO_ABERTO'] = bases_claro_operacional['SALDO_ABERTO'].str.replace('.', '').str.replace(',', '.').astype(float)
bases_claro_operacional['BASE'] = bases_claro_operacional['BASE'].apply(lambda x: x.replace('Claro MÃ\x83Â³vel','Claro Móvel'))


#### Filtrando apenas as colunas uteis para uma tabela dimensão
bases_claro_operacional = bases_claro_operacional[['CPF','DEVEDOR','TELEFONE','ATRASO','SALDO_ABERTO','BASE','TELEFONE_HIGIENIZADO']]

bases_claro_operacional['TELEFONE'] = bases_claro_operacional['TELEFONE'].str.strip()
bases_claro_operacional['TELEFONE_HIGIENIZADO'] = bases_claro_operacional['TELEFONE_HIGIENIZADO'].str.strip()

In [ ]:
#### Apartando as bases para termos a base em linhas unicas e nao ter o mesmo telefone na base higienizada e contratante.
#### Removendo os duplicados após essa separação.
base_contratatante = bases_claro_operacional[['CPF','DEVEDOR','TELEFONE','ATRASO','SALDO_ABERTO','BASE']]
base_contratatante = base_contratatante.drop_duplicates(['CPF','DEVEDOR','TELEFONE','ATRASO','BASE'])

base_higienizada = bases_claro_operacional[['CPF','DEVEDOR','TELEFONE_HIGIENIZADO','ATRASO','SALDO_ABERTO','BASE']]
base_higienizada = base_higienizada.drop_duplicates(['CPF','DEVEDOR','TELEFONE_HIGIENIZADO','ATRASO','BASE'])

In [ ]:
### TRAZENDO OS STATUS DE TELEFONIA, MARCANDO SE É ALTO OU BAIXO ALO
base_contratatante = pd.merge(base_contratatante, telefones_validados, right_on=['dddtelefone'], left_on=['TELEFONE'], how="left").loc[:,['CPF','DEVEDOR','TELEFONE','ATRASO','SALDO_ABERTO','BASE','STATUS_TEL']]
base_higienizada = pd.merge(base_higienizada, telefones_validados, right_on=['dddtelefone'], left_on=['TELEFONE_HIGIENIZADO'], how="left").loc[:,['CPF','DEVEDOR','TELEFONE_HIGIENIZADO','ATRASO','SALDO_ABERTO','BASE','STATUS_TEL']]

base_contratatante['STATUS_TEL'] = base_contratatante['STATUS_TEL'].fillna('nao_validado')
base_higienizada['STATUS_TEL'] = base_higienizada['STATUS_TEL'].fillna('nao_validado')


In [ ]:
### TRAZENDO AS DISCAGENS PARA A BASE
base_contratatante = pd.merge(base_contratatante, discagens_claro_full, right_on=['TELEFONE'], left_on=['TELEFONE'], how="left")

### TRAZENDO A BASE QUE ESTÁ EM BLACKLIST
base_contratatante = pd.merge(base_contratatante, blacklist, right_on=['TEL_BLACKLIST'], left_on=['TELEFONE'], how="left")

base_contratatante = base_contratatante[['CPF', 'DEVEDOR', 'TELEFONE', 'ATRASO', 'SALDO_ABERTO', 'BASE','STATUS_TEL', 'DISCADO_SIM', 'ATENDIDO_SIM', 'DTMF_SIM','DISCADO_UNICO', 'ATENDIDO_UNICO', 'BLACKLIST']]

# Contar os caracteres na coluna 'TELEFONE'
base_contratatante['NUM_CARACTERES'] = base_contratatante['TELEFONE'].apply(len)

# Criar a coluna 'TIPO_TEL' com base no número de caracteres
base_contratatante['TIPO_TEL'] = base_contratatante['NUM_CARACTERES'].apply(lambda x: 'CELULAR' if x == 11 else 'FIXO' if x == 10 else 'UNKNOWN')

# Remover a coluna auxiliar 'NUM_CARACTERES'
base_contratatante = base_contratatante.drop(columns=['NUM_CARACTERES'])

base_contratatante[['DISCADO_SIM','ATENDIDO_SIM','DTMF_SIM','DISCADO_UNICO','ATENDIDO_UNICO','BLACKLIST']] = base_contratatante[['DISCADO_SIM','ATENDIDO_SIM','DTMF_SIM','DISCADO_UNICO','ATENDIDO_UNICO','BLACKLIST']].fillna(0)

# Criando coluna Faixa de atraso com suas respectivas faixas
base_contratatante.loc[base_contratatante.ATRASO.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
base_contratatante.loc[base_contratatante.ATRASO.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
base_contratatante.loc[base_contratatante.ATRASO.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
base_contratatante.loc[base_contratatante.ATRASO.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
base_contratatante.loc[base_contratatante.ATRASO.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

### Removendo duplicatas por colunas chaves que nao devem se repetir
base_contratatante = base_contratatante.drop_duplicates(['CPF','TELEFONE','BASE','Faixa_de_Atraso']).reset_index(drop=True)



############################################## AGRUPANDO O ESTOQUE POR CARTEIRA E POR FAIXA DE ATRASO 
cpf_unico_por_faixa_de_atraso = base_contratatante[['CPF','BASE',"Faixa_de_Atraso"]].drop_duplicates(['CPF','BASE',"Faixa_de_Atraso"]).groupby(['BASE','Faixa_de_Atraso']).count().reset_index().rename(columns={'CPF':'CPF_UNICO_POR_FAIXA_ATRASO'})
cpf_unico_por_carteira = base_contratatante[['CPF','BASE']].drop_duplicates(['CPF','BASE']).groupby(['BASE']).count().reset_index().rename(columns={'CPF':'CPF_UNICO_POR_CARTEIRA'})

total_de_estoque = pd.merge(cpf_unico_por_faixa_de_atraso, cpf_unico_por_carteira, right_on=['BASE'], left_on=['BASE'], how="left")
total_de_estoque['CPF_UNICO_POR_CARTEIRA'] = total_de_estoque['CPF_UNICO_POR_CARTEIRA'][total_de_estoque.groupby('BASE')['CPF_UNICO_POR_CARTEIRA'].rank(method='first') == 1]
total_de_estoque = total_de_estoque.fillna('')
total_de_estoque['CPF_UNICO_POR_CARTEIRA'] = total_de_estoque['CPF_UNICO_POR_CARTEIRA'].astype(str).apply(lambda x: x.replace('.0',''))
total_de_estoque = total_de_estoque.rename(columns={'Faixa_de_Atraso':'PERFIL'})


#### SEPARANDO OS CPFS QUE FORAM PENETRADOS CONTRATANTES
penetracao_cpf_contratante = base_contratatante.query("ATENDIDO_SIM >= 1 ")[['BASE','Faixa_de_Atraso','CPF','ATENDIDO_SIM']]
penetracao_cpf_contratante = penetracao_cpf_contratante.assign(CPF_PENETRADO = 1)

In [ ]:
### TRAZENDO AS DISCAGENS PARA A BASE
base_higienizada = pd.merge(base_higienizada, discagens_claro_full, right_on=['TELEFONE'], left_on=['TELEFONE_HIGIENIZADO'], how="left")

### TRAZENDO A BASE QUE ESTÁ EM BLACKLIST
base_higienizada = pd.merge(base_higienizada, blacklist, right_on=['TEL_BLACKLIST'], left_on=['TELEFONE_HIGIENIZADO'], how="left")

base_higienizada = base_higienizada[['CPF', 'DEVEDOR', 'TELEFONE_HIGIENIZADO', 'ATRASO', 'SALDO_ABERTO', 'BASE','STATUS_TEL', 'DISCADO_SIM', 'ATENDIDO_SIM', 'DTMF_SIM','DISCADO_UNICO', 'ATENDIDO_UNICO', 'BLACKLIST']]

base_higienizada = base_higienizada.query("TELEFONE_HIGIENIZADO.notnull()")


# Contar os caracteres na coluna 'TELEFONE'
base_higienizada['NUM_CARACTERES'] = base_higienizada['TELEFONE_HIGIENIZADO'].apply(len)

# Criar a coluna 'TIPO_TEL' com base no número de caracteres
base_higienizada['TIPO_TEL'] = base_higienizada['NUM_CARACTERES'].apply(lambda x: 'CELULAR' if x == 11 else 'FIXO' if x == 10 else 'UNKNOWN')

# Remover a coluna auxiliar 'NUM_CARACTERES'
base_higienizada = base_higienizada.drop(columns=['NUM_CARACTERES'])

base_higienizada[['DISCADO_SIM','ATENDIDO_SIM','DTMF_SIM','DISCADO_UNICO','ATENDIDO_UNICO','BLACKLIST']] = base_higienizada[['DISCADO_SIM','ATENDIDO_SIM','DTMF_SIM','DISCADO_UNICO','ATENDIDO_UNICO','BLACKLIST']].fillna(0)

# Criando coluna Faixa de atraso com suas respectivas faixas
base_higienizada.loc[base_higienizada.ATRASO.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
base_higienizada.loc[base_higienizada.ATRASO.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
base_higienizada.loc[base_higienizada.ATRASO.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
base_higienizada.loc[base_higienizada.ATRASO.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
base_higienizada.loc[base_higienizada.ATRASO.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

base_higienizada = base_higienizada.drop_duplicates(['CPF','TELEFONE_HIGIENIZADO','BASE','Faixa_de_Atraso']).reset_index(drop=True)



#### SEPARANDO OS CPFS QUE FORAM PENETRADOS HIGIENIZADOS
penetracao_cpf_higienizada = base_higienizada.query("ATENDIDO_SIM >= 1 ")[['BASE','Faixa_de_Atraso','CPF','ATENDIDO_SIM']]
penetracao_cpf_higienizada = penetracao_cpf_higienizada.assign(CPF_PENETRADO = 1)


### Agrupamentos

In [ ]:
## AGRUPAMENTOS

### Agrupando discagens 
group_base_contratatante = base_contratatante.groupby(['BASE','Faixa_de_Atraso']).agg({
        'TELEFONE': 'count',
        'DISCADO_SIM':'sum',
        'DISCADO_UNICO':'sum',
        'ATENDIDO_SIM': 'sum',
        'ATENDIDO_UNICO':'sum',
        'DTMF_SIM':'sum',
        'BLACKLIST':'sum'}).reset_index()

### criando alguns parametros necessários sobre discagem.
group_base_contratatante['%TRAB'] = (group_base_contratatante['DISCADO_UNICO'] / group_base_contratatante['TELEFONE']* 100).round(2)
group_base_contratatante['%ATEND_UNICO'] =(group_base_contratatante['ATENDIDO_UNICO'] / group_base_contratatante['DISCADO_UNICO']*100).round(2)
group_base_contratatante['SPIN'] = (group_base_contratatante['DISCADO_SIM'] / group_base_contratatante['TELEFONE'].round(2)).round(2)
group_base_contratatante['%ALÔ'] = ((group_base_contratatante['ATENDIDO_SIM'] / group_base_contratatante['DISCADO_SIM'])* 100).round(2)
group_base_contratatante['%BLACKLIST'] = ((group_base_contratatante['BLACKLIST'] / group_base_contratatante['TELEFONE'])* 100).round(2)


group_base_contratatante[["DISCADO_SIM","ATENDIDO_SIM","DTMF_SIM","DISCADO_UNICO","ATENDIDO_UNICO","BLACKLIST"]] = group_base_contratatante[["DISCADO_SIM","ATENDIDO_SIM","DTMF_SIM","DISCADO_UNICO","ATENDIDO_UNICO","BLACKLIST"]].astype('str')

group_base_contratatante['DISCADO_SIM'] = group_base_contratatante['DISCADO_SIM'].apply(lambda x: x.replace('.0',''))
group_base_contratatante['ATENDIDO_SIM'] = group_base_contratatante['ATENDIDO_SIM'].apply(lambda x: x.replace('.0',''))
group_base_contratatante['DTMF_SIM'] = group_base_contratatante['DTMF_SIM'].apply(lambda x: x.replace('.0',''))
group_base_contratatante['DISCADO_UNICO'] = group_base_contratatante['DISCADO_UNICO'].apply(lambda x: x.replace('.0',''))
group_base_contratatante['ATENDIDO_UNICO'] = group_base_contratatante['ATENDIDO_UNICO'].apply(lambda x: x.replace('.0',''))
group_base_contratatante['BLACKLIST'] = group_base_contratatante['BLACKLIST'].apply(lambda x: x.replace('.0',''))

group_base_contratatante = group_base_contratatante.rename(columns={'TELEFONE':'TEL EPS','%ALÔ':'%ALÔ EPS' ,'%TRAB':'%TRAB EPS','%BLACKLIST':'%BLACKLIST EPS', 'DISCADO_SIM':'DISCADO EPS', 'DISCADO_UNICO':'DISC UNICO EPS', 'ATENDIDO_SIM':'ATENDIDO EPS','ATENDIDO_UNICO':'ATEND UNICO EPS','DTMF_SIM':'DTMF EPS','Faixa_de_Atraso':'PERFIL','%ATEND_UNICO':'%ATEND UNICO EPS','SPIN':'SPIN EPS'})
group_base_contratatante = group_base_contratatante[['BASE','PERFIL','TEL EPS','DISCADO EPS','ATENDIDO EPS','%ALÔ EPS','%ATEND UNICO EPS','%TRAB EPS','%BLACKLIST EPS','DTMF EPS','SPIN EPS']]


In [ ]:
## AGRUPAMENTOS

### Agrupando discagens 
group_base_higienizada = base_higienizada.groupby(['BASE','Faixa_de_Atraso']).agg({
        'TELEFONE_HIGIENIZADO': 'count',
        'DISCADO_SIM':'sum',
        'DISCADO_UNICO':'sum',
        'ATENDIDO_SIM': 'sum',
        'ATENDIDO_UNICO':'sum',
        'DTMF_SIM':'sum',
        'BLACKLIST':'sum'}).reset_index()

### criando alguns parametros necessários sobre discagem.
group_base_higienizada['%TRAB'] = (group_base_higienizada['DISCADO_UNICO'] / group_base_higienizada['TELEFONE_HIGIENIZADO']* 100).round(2)
group_base_higienizada['%ATEND_UNICO'] =(group_base_higienizada['ATENDIDO_UNICO'] / group_base_higienizada['DISCADO_UNICO']*100).round(2)
group_base_higienizada['SPIN'] = (group_base_higienizada['DISCADO_SIM'] / group_base_higienizada['TELEFONE_HIGIENIZADO'].round(2)).round(2)
group_base_higienizada['%ALÔ'] = ((group_base_higienizada['ATENDIDO_SIM'] / group_base_higienizada['DISCADO_SIM'])* 100).round(2)
group_base_higienizada['%BLACKLIST'] = ((group_base_higienizada['BLACKLIST'] / group_base_higienizada['TELEFONE_HIGIENIZADO'])* 100).round(2)


group_base_higienizada[["DISCADO_SIM","ATENDIDO_SIM","DTMF_SIM","DISCADO_UNICO","ATENDIDO_UNICO","BLACKLIST"]] = group_base_higienizada[["DISCADO_SIM","ATENDIDO_SIM","DTMF_SIM","DISCADO_UNICO","ATENDIDO_UNICO","BLACKLIST"]].astype('str')

group_base_higienizada['DISCADO_SIM'] = group_base_higienizada['DISCADO_SIM'].apply(lambda x: x.replace('.0',''))
group_base_higienizada['ATENDIDO_SIM'] = group_base_higienizada['ATENDIDO_SIM'].apply(lambda x: x.replace('.0',''))
group_base_higienizada['DTMF_SIM'] = group_base_higienizada['DTMF_SIM'].apply(lambda x: x.replace('.0',''))
group_base_higienizada['DISCADO_UNICO'] = group_base_higienizada['DISCADO_UNICO'].apply(lambda x: x.replace('.0',''))
group_base_higienizada['ATENDIDO_UNICO'] = group_base_higienizada['ATENDIDO_UNICO'].apply(lambda x: x.replace('.0',''))
group_base_higienizada['BLACKLIST'] = group_base_higienizada['BLACKLIST'].apply(lambda x: x.replace('.0',''))

group_base_higienizada = group_base_higienizada.rename(columns={'TELEFONE_HIGIENIZADO':'TEL HIGI','%ALÔ':'%ALÔ HIGI' ,'%TRAB':'%TRAB HIGI','%BLACKLIST':'%BLACKLIST HIGI', 'DISCADO_SIM':'DISCADO HIGI', 'DISCADO_UNICO':'DISC UNICO HIGI', 'ATENDIDO_SIM':'ATENDIDO HIGI','ATENDIDO_UNICO':'ATEND UNICO HIGI','DTMF_SIM':'DTMF HIGI','Faixa_de_Atraso':'PERFIL','%ATEND_UNICO':'%ATEND UNICO HIGI','SPIN':'SPIN HIGI'})

group_base_higienizada = group_base_higienizada[['BASE','PERFIL','TEL HIGI','DISCADO HIGI','ATENDIDO HIGI','%ALÔ HIGI','%ATEND UNICO HIGI','%TRAB HIGI','%BLACKLIST HIGI','DTMF HIGI','SPIN HIGI']]


In [ ]:
### AGRUPANDO AS PENETRAÇÕES POR CPF
group_penetração_por_cpf = pd.concat([penetracao_cpf_contratante,penetracao_cpf_higienizada]).drop_duplicates(['Faixa_de_Atraso','CPF']).groupby(['BASE','Faixa_de_Atraso'])['CPF_PENETRADO'].count().reset_index()
group_penetração_por_cpf = group_penetração_por_cpf.rename(columns={'Faixa_de_Atraso':'PERFIL'})

In [ ]:
#### Analise final
group_final = pd.merge(group_base_contratatante, group_base_higienizada, right_on=['BASE','PERFIL'], left_on=['BASE','PERFIL'], how="left")

## TRAZENDO O TOTAL DE ESTOQUE e CPF PENETRADO
group_final = pd.merge(group_final, total_de_estoque, right_on=['BASE','PERFIL'], left_on=['BASE','PERFIL'], how="left")

group_final = pd.merge(group_final, group_penetração_por_cpf, right_on=['BASE','PERFIL'], left_on=['BASE','PERFIL'], how="left")[['BASE','CPF_UNICO_POR_CARTEIRA', 'PERFIL','CPF_UNICO_POR_FAIXA_ATRASO','CPF_PENETRADO', 'TEL EPS', 'DISCADO EPS', 'ATENDIDO EPS', '%ALÔ EPS',
       '%ATEND UNICO EPS', '%TRAB EPS', '%BLACKLIST EPS', 'DTMF EPS',
       'SPIN EPS', 'TEL HIGI', 'DISCADO HIGI', 'ATENDIDO HIGI', '%ALÔ HIGI',
       '%ATEND UNICO HIGI', '%TRAB HIGI', '%BLACKLIST HIGI', 'DTMF HIGI',
       'SPIN HIGI']].rename(columns={'CPF_UNICO_POR_CARTEIRA':'CPF_UNICO','CPF_UNICO_POR_FAIXA_ATRASO':'CPF_UNICO_ATR'})

In [ ]:
### Salvando em csv o resultado de discagens
group_final.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\BASES_E_DISCAGENS_TELES/dados_claro_julho.csv",sep=';',index=False,encoding='iso-8859-1')

In [ ]:
# Certifique-se de que o dataframe 'group_final' esteja carregado no seu ambiente
# Por exemplo, se estiver carregando de um arquivo CSV:
# group_final = pd.read_csv('caminho/para/seu/arquivo.csv')

# Defina as colunas que precisam ser destacadas
eps_columns = ['TEL EPS','DISCADO EPS','ATENDIDO EPS','%ALÔ EPS','%ATEND UNICO EPS','%TRAB EPS','%BLACKLIST EPS','DTMF EPS','SPIN EPS']
higi_columns = ['TEL HIGI','DISCADO HIGI','ATENDIDO HIGI','%ALÔ HIGI','%ATEND UNICO HIGI','%TRAB HIGI','%BLACKLIST HIGI','DTMF HIGI','SPIN HIGI']


# Caminho para salvar o arquivo Excel
excel_path = r'C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\BASES_E_DISCAGENS_TELES/dados_claro_julho.xlsx'

# Salve o dataframe no Excel com formatação
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    group_final.to_excel(writer, sheet_name='Sheet1', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # Formatos de cor em hexadecimal (amarelo suave e verde suave) com borda
    format_eps = workbook.add_format({'border': 1})  # LemonChiffon
    format_higi = workbook.add_format({'border': 1})  # LightGreen
    ini_cols = workbook.add_format({'border': 1})
    
    
    # Aplique formatos para colunas específicas e linhas 2 a 13 com borda
    for col in eps_columns:
        col_idx = group_final.columns.get_loc(col)
        worksheet.conditional_format(0, col_idx, 19, col_idx, {'type': 'no_blanks', 'format': format_eps})
        
    for col in higi_columns:
        col_idx = group_final.columns.get_loc(col)
        worksheet.conditional_format(0, col_idx, 19, col_idx, {'type': 'no_blanks', 'format': format_higi})

### Acordos

In [ ]:
import pandas as pd
import warnings
import glob

# Configurações e avisos
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

# Definir colunas necessárias
colunas_necessarias = ['Data do Acordo','Negociador','Acordo', 'Carteira', 'Negociador', 'Senha', 'Atraso']

# Definir caminho para os arquivos
caminho = r"R:\TI\MIS SUP\ACORDOS_CLARO_NET_GEVENUE\INTERSIC"
arquivos = glob.glob(caminho + "/*.txt")

# Inicializar uma lista para armazenar DataFrames
dfList = []

# Ler e processar cada arquivo
for arquivo in arquivos:
    # Ler o arquivo inteiro de uma vez
    df = pd.read_csv(arquivo, encoding='iso-8859-1', sep=";",usecols=colunas_necessarias)
    # Adicionar o DataFrame à lista
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
acordos_intersic = pd.concat(dfList, sort=False, ignore_index=True)

# Função para corrigir strings mal codificadas
def correct_encoding(text):
    if isinstance(text, str):
        try:
            return text.encode('latin1').decode('utf-8')
        except UnicodeDecodeError:
            return text
    return text

# Aplicar a função de correção a todas as colunas do DataFrame que são do tipo object (strings)
for col in acordos_intersic.select_dtypes(include=[object]).columns:
    acordos_intersic['Negociador'] = acordos_intersic['Negociador'].apply(correct_encoding)



# acordos_intersic = pd.read_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\ACORDOS CLARO E NET\Mis 15_06\Claro\Intersic/Acordosporperíodo_19592024115934217263AD9D0F4F.txt", encoding='iso-8859-1', sep=";",usecols=colunas_necessarias)
# filtrando oq é atn
acordos_intersic = acordos_intersic[acordos_intersic['Negociador'].str.contains('ATN')]

# Remover o conteúdo entre parênteses na coluna 'Negociador'
acordos_intersic['Negociador'] = acordos_intersic['Negociador'].str.replace(r'\s*\([^)]*\)', '', regex=True)


# Definir a função de ajuste
def ajustar_carteira(carteira):
    if 'TV' in carteira:
        return 'Claro TV'
    elif 'FIXO' in carteira:
        return 'Claro Fixo'
    else:
        return 'Claro Móvel'

# Aplicar a função à coluna 'Carteira'
acordos_intersic['Carteira'] = acordos_intersic['Carteira'].apply(ajustar_carteira)
acordos_intersic = acordos_intersic.assign(ACORDO = 1)
acordos_intersic['Data do Acordo'] = acordos_intersic['Data do Acordo'].str[:10]

# Contar a ocorrência de cada contrato
contagem_contratos = acordos_intersic['Senha'].value_counts()
# Criar a coluna 'ACORDO_UNICO'
acordos_intersic['ACORDO_UNICO'] = acordos_intersic['Senha'].apply(lambda x: 1 if contagem_contratos[x] == 1 else 0)

In [ ]:
#############################CARREGANDO OS ACORDOS DA GEVENUE

colunas_necessarias_gevenue = ['Data do Acordo ','Negociador','ID Acordo','Produto','Id Gevenue','Aging','Agência']

# Definir caminho para os arquivos
caminho = r"R:\TI\MIS SUP\ACORDOS_CLARO_NET_GEVENUE\GEVENUE"
arquivos = glob.glob(caminho + "/*.csv")

# Inicializar uma lista para armazenar DataFrames
dfList = []

# Ler e processar cada arquivo
for arquivo in arquivos:
    # Ler o arquivo inteiro de uma vez
    df = pd.read_csv(arquivo, encoding='utf-8', sep="|",usecols=colunas_necessarias_gevenue)
    # Adicionar o DataFrame à lista
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
acordos_gevenue = pd.concat(dfList, sort=False, ignore_index=True)
acordos_gevenue = acordos_gevenue[acordos_gevenue['Agência'].str.contains('ATN')]
acordos_gevenue
acordos_gevenue.rename(columns={'ID Acordo':'Acordo','Produto':'Carteira','Id Gevenue':'Senha','Aging':'Atraso','Data do Acordo ':'Data do Acordo'},inplace=True)

# Definir o mapeamento de valores para substituição
replace_dict = {
    'ClaroMovel': 'GEVENUE CLARO MOVEL',
    'NetTvVirtua': 'GEVENUE NET',
    'NetFone': 'GEVENUE NET',
    'ClaroFixo': 'GEVENUE CLARO FIXO',
}

# Aplicar o replace na coluna 'Carteira'
acordos_gevenue['Carteira'] = acordos_gevenue['Carteira'].replace(replace_dict)

acordos_gevenue = acordos_gevenue.assign(ACORDO = 1)

# Contar a ocorrência de cada contrato
contagem_contratos = acordos_gevenue['Senha'].value_counts()
# Criar a coluna 'ACORDO_UNICO'
acordos_gevenue['ACORDO_UNICO'] = acordos_gevenue['Senha'].apply(lambda x: 1 if contagem_contratos[x] == 1 else 0)
acordos_gevenue = acordos_gevenue[['Acordo', 'Carteira','Negociador', 'Senha', 'Data do Acordo', 'Atraso', 'ACORDO',  'ACORDO_UNICO']]

In [ ]:
### CONCATENANDO OS ACORDOS DOS SISTEMAS INTERSIC E GEVENUE
acordos_teles = pd.concat([acordos_intersic,acordos_gevenue])
acordos_teles['Atraso'] = acordos_teles['Atraso'].astype('int16')

# Criando coluna Faixa de atraso com suas respectivas faixas
acordos_teles.loc[acordos_teles.Atraso.between(-9999, -1),    'Faixa_de_Atraso'] = "Atraso (-1 a -9999)"
acordos_teles.loc[acordos_teles.Atraso.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
acordos_teles.loc[acordos_teles.Atraso.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
acordos_teles.loc[acordos_teles.Atraso.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
acordos_teles.loc[acordos_teles.Atraso.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
acordos_teles.loc[acordos_teles.Atraso.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

acordos_teles = acordos_teles.drop(columns=['Senha','Atraso','Acordo']).loc[:,['Data do Acordo','Faixa_de_Atraso','Carteira','Negociador','ACORDO','ACORDO_UNICO']].rename(columns={'Carteira':'BASE'})
# Removendo acentos
acordos_teles['Negociador'] = acordos_teles['Negociador'].apply(lambda x: unidecode(x))
acordos_teles['Negociador'] = acordos_teles['Negociador'].astype(str).str.strip()

### BAIXANDO O ANALITICO DOS ACORDOS
acordos_teles.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\ACORDOS_AGENTES/acordos_teles_agentes_julho.csv",index=False,sep=';',encoding='iso-8859-1')


#### Acionamentos

In [ ]:
# Definir colunas necessárias
colunas_necessarias = ['TIPO_PRODUTO','USU_NOME','DEV_CPF','ACN_DESCRICAO','DEA_DATA','ATRASO']

# Definir caminho para os arquivos
caminho = r"R:\TI\MIS SUP\ACIONAMENTOS_CLARO_NET_GEVENUE\INTERSIC"
arquivos = glob.glob(caminho + "/*.txt")

# Inicializar uma lista para armazenar DataFrames
dfList = []

# Ler e processar cada arquivo
for arquivo in arquivos:
    # Ler o arquivo inteiro de uma vez
    df = pd.read_csv(arquivo, encoding='iso-8859-1', sep=";",error_bad_lines=False, usecols=colunas_necessarias,dtype={'DEV_CPF':'str'})
    # Adicionar o DataFrame à lista
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
acionamento_intersic = pd.concat(dfList, sort=False, ignore_index=True)

# filtrando oq é atn
acionamento_intersic = acionamento_intersic[acionamento_intersic['USU_NOME'].str.contains('ATN')]
# Remover o conteúdo entre parênteses na coluna 'Negociador'
acionamento_intersic['USU_NOME'] = acionamento_intersic['USU_NOME'].str.replace(r'\s*\([^)]*\)', '', regex=True)

acionamento_intersic['DEA_DATA'] = acionamento_intersic['DEA_DATA'].str[:10]


# Criando coluna Faixa de atraso com suas respectivas faixas
acionamento_intersic['ATRASO'] = acionamento_intersic['ATRASO'].astype('int64')
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
acionamento_intersic.loc[acionamento_intersic.ATRASO.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

## Ajustando layout
acionamento_intersic = acionamento_intersic.drop(columns=['ATRASO'])[['DEA_DATA','TIPO_PRODUTO',	'USU_NOME',	'DEV_CPF',	'ACN_DESCRICAO','Faixa_de_Atraso']]

In [ ]:
# Definir colunas necessárias
colunas_necessarias = ['Nome do UsuÃ¡rio','CPF/CNPJ','Aging','Data do Acionamento','Acionamento Feito','Valor Net','Valor Embratel','Valor Movel','Valor Claro TV','Valor Claro Fixo']

# Definir caminho para os arquivos
caminho = r"R:\TI\MIS SUP\ACIONAMENTOS_CLARO_NET_GEVENUE\GEVENUE"
arquivos = glob.glob(caminho + "/*.csv")

# Inicializar uma lista para armazenar DataFrames
dfList = []

# Ler e processar cada arquivo
for arquivo in arquivos:
    # Ler o arquivo inteiro de uma vez
    df = pd.read_csv(arquivo, encoding='iso-8859-1', sep="|",error_bad_lines=False, usecols=colunas_necessarias)
    # Adicionar o DataFrame à lista
    dfList.append(df)

# Concatenar todos os DataFrames em um único DataFrame
acionamento_gevenue = pd.concat(dfList, sort=False, ignore_index=True)

## Remover a aspas do cpf
acionamento_gevenue['CPF/CNPJ'] = acionamento_gevenue['CPF/CNPJ'].str.replace(r'\D', '', regex=True)


# Função para determinar os produtos em uma lista
def determinar_produto(row):
    produtos = []
    if pd.notna(row['Valor Net']):
        produtos.append('GEVENUE NET')
    if pd.notna(row['Valor Embratel']):
        produtos.append('GEVENUE NET')
    if pd.notna(row['Valor Movel']):
        produtos.append('GEVENUE CLARO MOVEL')
    if pd.notna(row['Valor Claro TV']):
        produtos.append('GEVENUE CLARO TV')
    if pd.notna(row['Valor Claro Fixo']):
        produtos.append('GEVENUE CLARO FIXO')
    return produtos if produtos else None

# Aplicar a função ao DataFrame
acionamento_gevenue['PRODUTO'] = acionamento_gevenue.apply(determinar_produto, axis=1)

# Explodir a coluna PRODUTO para criar linhas separadas
acionamento_gevenue = acionamento_gevenue.explode('PRODUTO')

## Removendo duplicatas para nao trazer o produto net duas vezes desnecessáriamente
acionamento_gevenue = acionamento_gevenue.drop_duplicates(['Data do Acionamento','CPF/CNPJ','PRODUTO']).reset_index(drop=True)

# Criando coluna Faixa de atraso com suas respectivas faixas
acionamento_gevenue['Aging'] = acionamento_gevenue['Aging'].astype('int64')
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(0, 90),    'Faixa_de_Atraso'] = "PDD (0 A 90)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(91, 360),    'Faixa_de_Atraso'] = "Atraso (91 a 360)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(361, 1080),    'Faixa_de_Atraso'] = "Atraso (361 a 1080)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(1081, 1800),    'Faixa_de_Atraso'] = "Atraso (1081 a 1800)"
acionamento_gevenue.loc[acionamento_gevenue.Aging.between(1801, 999999),    'Faixa_de_Atraso'] = "Atraso (Acima 1801)"

### ajustando layout
acionamento_gevenue = acionamento_gevenue.rename(columns={'Nome do UsuÃ¡rio':'USU_NOME','CPF/CNPJ':'DEV_CPF','Data do Acionamento':'DEA_DATA','Acionamento Feito':'ACN_DESCRICAO','PRODUTO':'TIPO_PRODUTO'})\
[['DEA_DATA','TIPO_PRODUTO',	'USU_NOME',	'DEV_CPF',	'ACN_DESCRICAO','Faixa_de_Atraso']]



### Juntando os acionamentos e marcando oq é cpc
acionamentos_teles = pd.concat([acionamento_intersic,acionamento_gevenue])
acionamentos_teles = acionamentos_teles.assign(ACIONAMENTO = 1)

lista_cpc = ['ALEGA PROCESSO JUDICIAL',
'ALEGA DESEMPREGO',
'ALEGA FRAUDE',
'ESTÁ NEGOCIANDO COM O CREDOR',
'REENVIO DE BOLETO',
'CONTESTAÇÃO',
'ALEGA PAGAMENTO',
'NÃO RECONHECE A LINHA',
'PREVENTIVO REALIZADO',
'PROMESSA DE PAGAMENTO',
'SEM CONDIÇÕES FINANCEIRAS',
'NÃO CONCORDA COM NEGATIVAÇÃO SPC/SERASA',
'NÃO CONCORDA VALOR COBRADO',
'PROBLEMAS DE SAÚDE',
'RECUSA-SE A PAGAR',
'Acionamento Ext. - Alega Desemprego',
'Acionamento Ext. - Alega Fraude',
'Acionamento Ext. - NÃ£o concorda com valor cobrado',
'Acionamento Ext. - NÃ£o reconhece a linha',
'Acionamento Ext. - Preventivo realizado',
'Acionamento Ext. - Problema de saÃºde',
'Acionamento Ext. - Promessa de Pagamento',
'Acionamento Ext. - Sem CondiÃ§Ãµes Financeiras',
'Alega Pagamento',
'Alega Processo Judicial',
'ContestaÃ§Ã£o',
'EstÃ¡ negociando com o Credor',
'NÃ£o concorda com NegativaÃ§Ã£o SPC/SERASA',
'Recusa-se a pagar']

acionamentos_teles['CPC'] = np.where(acionamentos_teles['ACN_DESCRICAO'].isin(lista_cpc), 1, 0)

In [ ]:
acionamentos_teles.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\ACIONAMENTOS_TELEES/acionamentos_teles_julho.csv",index=False,sep=';',encoding='iso-8859-1')

### Agentes

In [ ]:

#################################################################################### CARREGANDO TODOS ATENDIMENTOS DO MES (AGENTE) #########################################################
caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES CLARO\CHAMADAS_EQUIPE_CLARO\JULHO_24\ATENDIDAS AGENTES JULHO"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    lista.append(arquivo)
    dfList = []
    for item in lista:   
        chunks = pd.read_csv(item, chunksize=1000000, sep=';', encoding='iso-8859-1', low_memory=False, dtype= {'CPF_AGENTE':str})
        for df in chunks:
            dfList.append(df)
# Criando DataFrame com concat e a lista criada a cima
atendidos_agentes = pd.concat(dfList, sort=False, ignore_index=False)
atendidos_agentes = atendidos_agentes.reset_index(drop=True)
for col in atendidos_agentes.select_dtypes(include=[object]).columns:
    atendidos_agentes['NOME_AGENTE'] = atendidos_agentes['NOME_AGENTE'].apply(correct_encoding)

atendidos_agentes = atendidos_agentes[['DATA','CPF_AGENTE','NOME_AGENTE','FILA','ATENDIDAS','PREDITIVO','URA','RECEPTIVO']]


# Removendo acentos
atendidos_agentes['NOME_AGENTE'] = atendidos_agentes['NOME_AGENTE'].apply(lambda x: unidecode(x))
atendidos_agentes['NOME_AGENTE'] = atendidos_agentes['NOME_AGENTE'].astype(str).str.strip()

atendidos_agentes.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\ATENDIMENTOS_AGENTES/atendidos_teles_agentes_julho.csv",index=False,sep=';',encoding='iso-8859-1')


#####

In [ ]:
from funcoes_claro import *

# Carregar o arquivo Excel com a coluna 'CPF_AGENTE' como string
info_agentes = pd.read_excel(r'R:\CARTEIRAS\COBRANÇA\CLARO COBRANCA\MAILINGS/LOGIN X NOME OPERAÇÃO.xlsx',   sheet_name='LOGINS',   dtype={'CPF_AGENTE': str})

# Verificar se a coluna 'CPF_AGENTE' está realmente como string
info_agentes['CPF AGENTE'] = info_agentes['CPF AGENTE'].astype(str)

# Caso haja valores numéricos convertidos, garantir a presença dos zeros à esquerda
info_agentes['CPF AGENTE'] = info_agentes['CPF AGENTE'].apply(lambda x: x.zfill(11))


###################################################################################
### ACORDOS DOS AGENTES MES VIGENTE 
###################################################################################
caminho = r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\ACORDOS_AGENTES"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    lista.append(arquivo)
    dfList = []
    for item in lista:   
        chunks = pd.read_csv(item, chunksize=1000000, sep=';', encoding='iso-8859-1', low_memory=False)
        for df in chunks:
            dfList.append(df)
# Criando DataFrame com concat e a lista criada a cima
acordos_agentes_vigente = pd.concat(dfList, sort=False, ignore_index=False)

# Criando uma coluna chamada FILA, para quando cruzar os acordos dos agentes POR FILA saber onde ele performou melhor, caso seja hibrido.
def definir_fila(base):
    if pd.notna(base):  # Verifica se a entrada não é NaN
        base_lower = base.lower()  # Converte a string para minúsculas
        if 'claro' in base_lower:
            return 'CLARO'
        elif 'net' in base_lower:
            return 'NET'
    return None

# Aplicando a função para criar a coluna FILA
acordos_agentes_vigente['FILA'] = acordos_agentes_vigente['BASE'].apply(definir_fila)

###################################################################################
### ATENDIMENTO DOS AGENTES MES VIGENTE 
###################################################################################
caminho = r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\ATENDIMENTOS_AGENTES"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    lista.append(arquivo)
    dfList = []
    for item in lista:   
        chunks = pd.read_csv(item, chunksize=1000000, sep=';', encoding='iso-8859-1', low_memory=False, dtype= {'CPF AGENTE':str})
        for df in chunks:
            dfList.append(df)
# Criando DataFrame com concat e a lista criada a cima
atendimentos_agentes_vigente = pd.concat(dfList, sort=False, ignore_index=False)


###################################################################################
### CARREGANDO ATENDIMENTO DOS AGENTES MES VIGENTE
###################################################################################

caminho = r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\BASES_E_DISCAGENS_TELES"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    lista.append(arquivo)
    dfList = []
    for item in lista:   
        chunks = pd.read_csv(item, chunksize=1000000, sep=';', encoding='iso-8859-1', low_memory=False, dtype= {'CPF_UNICO':str})
        for df in chunks:
            dfList.append(df)
# Criando DataFrame com concat e a lista criada a cima
bases_e_discagens_vigente = pd.concat(dfList, sort=False, ignore_index=False)
bases_e_discagens_vigente['CPF_UNICO'] = bases_e_discagens_vigente['CPF_UNICO'].fillna('')



###################################################################################
### CARREGANDO OS ACIONAMENTOS DO MES VIGENTE
###################################################################################

caminho = r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE\ACIONAMENTOS_TELEES"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    lista.append(arquivo)
    dfList = []
    for item in lista:   
        chunks = pd.read_csv(item, chunksize=1000000, sep=';', encoding='iso-8859-1', low_memory=False, dtype= {'CPF_UNICO':str})
        for df in chunks:
            dfList.append(df)
# Criando DataFrame com concat e a lista criada a cima
acionamentos_vigente = pd.concat(dfList, sort=False, ignore_index=False)
acionamentos_vigente = acionamentos_vigente.rename(columns={'TIPO_PRODUTO':'BASE'})
# Aplicando a função para criar a coluna FILA
acionamentos_vigente['FILA'] = acionamentos_vigente['BASE'].apply(definir_fila)

In [ ]:
#### TRAZENDO A INFORMAÇÃO DE CPF E NOME DO AGENTE
acordos_agentes_vigente = pd.merge(acordos_agentes_vigente, info_agentes, right_on=['LOGIN'], left_on=['Negociador'], how="left").drop(columns={'LOGIN'})
acordos_agentes_vigente = acordos_agentes_vigente[['Data do Acordo','Faixa_de_Atraso','BASE','FILA','CPF AGENTE','NOME OPERADOR','Negociador','ACORDO','ACORDO_UNICO']]
acordos_agentes_vigente = acordos_agentes_vigente.rename(columns={'Negociador':'NOME_SISTEMA'})

atendimentos_agentes_vigente = pd.merge(atendimentos_agentes_vigente, info_agentes, right_on=['LOGIN'], left_on=['NOME_AGENTE'], how="left").drop(columns={'LOGIN','CPF_AGENTE'})
atendimentos_agentes_vigente = atendimentos_agentes_vigente[['DATA','FILA','CPF AGENTE','NOME OPERADOR','NOME_AGENTE','ATENDIDAS','PREDITIVO','URA','RECEPTIVO']]
atendimentos_agentes_vigente = atendimentos_agentes_vigente.rename(columns={'NOME_AGENTE':'NOME_SISTEMA'})

acionamentos_vigente = pd.merge(acionamentos_vigente, info_agentes, right_on=['LOGIN'], left_on=['USU_NOME'], how="left").drop(columns={'LOGIN'})
acionamentos_vigente = acionamentos_vigente[['DEA_DATA','BASE','Faixa_de_Atraso','FILA','CPF AGENTE','NOME OPERADOR','USU_NOME','DEV_CPF','ACN_DESCRICAO','ACIONAMENTO','CPC']]
acionamentos_vigente = acionamentos_vigente.rename(columns={'USU_NOME':'NOME_SISTEMA'})

#### AGRUPANDO OS ACORDOS E DISCAGENS DOS AGENTES

In [ ]:
gp_atendimentos_diarios = atendimentos_agentes_vigente.groupby(['DATA','CPF AGENTE','NOME OPERADOR']).agg({
        'ATENDIDAS': 'count',
        'PREDITIVO':'sum',
        'URA':'sum',
        'RECEPTIVO': 'sum'}).reset_index()


gp_acordos_diarios = acordos_agentes_vigente.groupby(['Data do Acordo','CPF AGENTE','NOME OPERADOR']).agg({
        'ACORDO': 'count',
        'ACORDO_UNICO':'sum'}).reset_index()

########################### AGRUPAMENTO CONSOLIDADO, TODAS AS CHAMADAS E ACORDOS DO AGENTE INDEPENDENTE DA FILA
performance_agentes = pd.merge(gp_atendimentos_diarios, gp_acordos_diarios, right_on=['Data do Acordo','CPF AGENTE','NOME OPERADOR'], left_on=['DATA','CPF AGENTE','NOME OPERADOR'], how="left").drop(columns='Data do Acordo')

In [ ]:
gp_atendimentos_por_fila = atendimentos_agentes_vigente.groupby(['DATA','CPF AGENTE','NOME OPERADOR','FILA']).agg({
        'ATENDIDAS': 'count',
        'PREDITIVO':'sum',
        'URA':'sum',
        'RECEPTIVO': 'sum'}).reset_index()


gp_acordos_por_fila = acordos_agentes_vigente.groupby(['Data do Acordo','CPF AGENTE','NOME OPERADOR','FILA']).agg({
        'ACORDO': 'count',
        'ACORDO_UNICO':'sum'}).reset_index()

performance_agentes_por_fila = pd.merge(gp_atendimentos_por_fila, gp_acordos_por_fila, right_on=['Data do Acordo','CPF AGENTE','NOME OPERADOR','FILA'], left_on=['DATA','CPF AGENTE','NOME OPERADOR','FILA'], how="left")

In [ ]:
### Acionamento por agente
gp_acionamento_agente = acionamentos_vigente.groupby(['DEA_DATA','CPF AGENTE','NOME OPERADOR']).agg({
        'ACIONAMENTO': 'sum',
        'CPC':'sum'}).reset_index()



### Acionamento por agente e fila
gp_acionamento_fila = acionamentos_vigente.groupby(['DEA_DATA','CPF AGENTE','NOME OPERADOR','FILA']).agg({
        'ACIONAMENTO': 'sum',
        'CPC':'sum'}).reset_index()


### Acionamento por peril e faixa de atraso
gp_acionamento_perfil = acionamentos_vigente.groupby(['BASE','Faixa_de_Atraso']).agg({
        'ACIONAMENTO': 'sum',
        'CPC':'sum'}).reset_index().rename(columns={'Faixa_de_Atraso':'PERFIL'})

In [ ]:
### trazendo os acionamentos para a performance do agente
performance_agentes = pd.merge(performance_agentes, gp_acionamento_agente, right_on=['DEA_DATA','CPF AGENTE','CPF AGENTE','NOME OPERADOR'], left_on=['DATA','CPF AGENTE','CPF AGENTE','NOME OPERADOR'], how="left").drop(columns='DEA_DATA')

### trazendo os acionamentos para a performance do agente por fila
performance_agentes_por_fila = performance_agentes_por_fila = pd.merge(performance_agentes_por_fila, gp_acionamento_fila,right_on=['DEA_DATA','CPF AGENTE','CPF AGENTE','NOME OPERADOR','FILA'], left_on=['DATA','CPF AGENTE','CPF AGENTE','NOME OPERADOR','FILA'], how="left").drop(columns=['Data do Acordo','DEA_DATA'])


#### AGRUPANDO OS ACORDOS E DISCAGENS DAS BASES CONSOLIDADAS

In [ ]:
########################### AGRUPAMENTO CONSOLIDADO, TODAS AS CHAMADAS E ACORDOS DO AGENTE POR FILA
gp_acordos_base_por_faixa_de_atraso = acordos_agentes_vigente.groupby(['BASE','Faixa_de_Atraso']).agg({
        'ACORDO': 'count',
        'ACORDO_UNICO':'sum'}).reset_index()   

bases_e_discagens_vigente = pd.merge(bases_e_discagens_vigente, gp_acordos_base_por_faixa_de_atraso, right_on=['BASE','Faixa_de_Atraso'], left_on=['BASE','PERFIL'], how="left")
bases_e_discagens_vigente[["ACORDO","ACORDO_UNICO"]] = bases_e_discagens_vigente[["ACORDO","ACORDO_UNICO"]].astype('str')
bases_e_discagens_vigente['ACORDO'] = bases_e_discagens_vigente['ACORDO'].apply(lambda x: x.replace('.0',''))
bases_e_discagens_vigente['ACORDO_UNICO'] = bases_e_discagens_vigente['ACORDO_UNICO'].apply(lambda x: x.replace('.0',''))
bases_e_discagens_vigente[['ACORDO','ACORDO_UNICO']] = bases_e_discagens_vigente[['ACORDO','ACORDO_UNICO']].apply(lambda x: x.replace('nan',0))
bases_e_discagens_vigente = bases_e_discagens_vigente.drop(columns='Faixa_de_Atraso')


### trazendo os acionamentos para o relatorio de base e discagens 
bases_e_discagens_vigente = pd.merge(bases_e_discagens_vigente, gp_acionamento_perfil, right_on=['BASE','PERFIL'], left_on=['BASE','PERFIL'], how="left")

In [ ]:
##### AJUSTANDO PRO EXCEL FICAR LIMPO
# Função para substituir pontos por vírgulas em colunas numéricas convertidas para strings
def replace_decimal_points(df, cols):
    for col in cols:
        df[col] = df[col].astype(str).str.replace('.', ',')

# Aplicar a função no DataFrame
bases_e_discagens_vigente_cols_to_replace = ['%ALÔ EPS', '%ATEND UNICO EPS','%TRAB EPS','%BLACKLIST EPS','SPIN EPS','%ALÔ HIGI','%ATEND UNICO HIGI','%TRAB HIGI','%BLACKLIST HIGI','SPIN HIGI']

replace_decimal_points(bases_e_discagens_vigente, bases_e_discagens_vigente_cols_to_replace)


performance_agentes_por_fila = performance_agentes_por_fila.fillna(0)
performance_agentes_por_fila[['ACORDO','ACORDO_UNICO','ACIONAMENTO','CPC']] = performance_agentes_por_fila[['ACORDO','ACORDO_UNICO','ACIONAMENTO','CPC']].astype('int16')

performance_agentes = performance_agentes.fillna(0)
performance_agentes[['ACORDO','ACORDO_UNICO','ACIONAMENTO','CPC']] = performance_agentes[['ACORDO','ACORDO_UNICO','ACIONAMENTO','CPC']].astype('int16')

In [ ]:
##### BAIXANDO TODOS OS AGRUPAMENTOS RESULTADOS FINAIS
performance_agentes.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE/performance_agentes_julho.csv",sep=';',index=False)
performance_agentes_por_fila.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE/performance_agentes_por_fila_julho.csv",sep=';',index=False)
bases_e_discagens_vigente.to_csv(r"C:\Users\jorgean.bomfim\Desktop\SCRIPTS EM PRODUÇÃO\CRM CLARO E NET\ANALISES_CLARO\DADOS TELES MES VIGENTE/performance_base_julho.csv",sep=';',index=False, encoding='iso-8859-1')